<a href="https://colab.research.google.com/github/sebastianduhe/Introduccion-a-la-IA/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega final curso de Inteligencia Artificial para las Ciencias y las Ingenierías



### Carga de librerias

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [ ]:
import seaborn as sns
import os
import json
import joblib

!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


### Obtenemos los datos

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']='/content/'

data = {"username":"baelfire","key":"b860190c7605c03bb89762805f39fb7f"}
with open('kaggle.json','w') as file:
  json.dump(data, file, indent=4)

## Descargamos base de datos

!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 939MB/s]


In [ ]:
## Descomprimimos

!unzip \*.zip && rm *.zip
!wc *.csv

Archive:  udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               
   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


### Leemos la base de datos

In [ ]:
## Leer la base de datos

bdtrain = pd.read_csv("train.csv")
print ("shape of loaded dataframe", bdtrain.shape)

shape of loaded dataframe (692500, 21)


In [ ]:
## Leer la base de datos

bdtest = pd.read_csv("test.csv")
print ("shape of loaded dataframe", bdtest.shape)

shape of loaded dataframe (296786, 20)


### Realizamos limpieza, reemplazo y normalizacion

In [ ]:
def limpdata(bd2):

  # Eliminamos variables repetidas
  def ecr(bd2):
    cols = bd2.columns
    colel = []
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            if bd2[cols[i]].equals(bd2[cols[j]]):
                colel.append(cols[j])
    return bd2.drop(columns=colel)
    bd2 = ecr(bd2)

    # Variables Categoricas
  vcat = [i for i in bd2.columns if not i in bd2._get_numeric_data()]
  print (vcat)

    # Valor mas repetido
  vc = bd2[vcat].mode().iloc[0]
  vc

  # Reemplazar datos faltantes con los mas repetidos
  for column in vcat:
    bd2[column].fillna(vc[column], inplace=True)

  # Modificar las columnas de Si y No por 1 y 0
  vbin = ['FAMI_TIENEINTERNET',
                 'FAMI_TIENELAVADORA',
                 'FAMI_TIENEAUTOMOVIL',
                 'ESTU_PAGOMATRICULAPROPIO',
                 'FAMI_TIENECOMPUTADOR']

  for col in vbin:
    bd2[col] = bd2[col].replace({'Si': 1, 'No': 0})
  # Normalizar otras variables con valores de 0, 1, ..., y así sucesivamente.

  hextra = {'0': 0,'Menos de 10 horas': 1,'Entre 11 y 20 horas': 2,'Entre 21 y 30 horas': 3, 'Más de 30 horas': 4}
  bd2["ESTU_HORASSEMANATRABAJA"] = bd2["ESTU_HORASSEMANATRABAJA"].replace(hextra)

  estfam = {'Sin Estrato': 0,'Estrato 1': 1,'Estrato 2': 2,'Estrato 3': 3, 'Estrato 4': 4, 'Estrato 5': 5, 'Estrato 6': 6}
  bd2["FAMI_ESTRATOVIVIENDA"] = bd2["FAMI_ESTRATOVIVIENDA"].replace(estfam)

  valmat = {'No pagó matrícula': 0, 'Menos de 500 mil': 1,
          'Entre 500 mil y menos de 1 millón': 2,'Entre 1 millón y menos de 2.5 millones': 3,
          'Entre 2.5 millones y menos de 4 millones': 4, 'Entre 4 millones y menos de 5.5 millones': 5,
          'Entre 5.5 millones y menos de 7 millones': 6, 'Más de 7 millones': 7}
  bd2["ESTU_VALORMATRICULAUNIVERSIDAD"] = bd2["ESTU_VALORMATRICULAUNIVERSIDAD"].replace(valmat)

  # Modificar la columna del Programa academico
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL EN ', '')
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL DE ', '')
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].str.replace('PROFESIONAL ', '')
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].str.replace('PROGRAMA ', '')

  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INGENIERIA' if x.startswith('INGE') else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'LICENCIATURA' if x.startswith('LIC') or x.startswith('Lic') else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ADMINISTRACION' if x.startswith('ADM')   else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CONTADURIA' if x.startswith('CONT')   else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'COMUNICACION' if x.startswith('COMU')   else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DEPORTE' if 'DEPORT'  in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'PSICOLOGIA' if 'PSIC' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TRABAJO SOCIAL' if 'TRABAJO SOCIAL' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DISEÑO' if 'DISE' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TERAPIA' if 'TERAP' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'GASTRONOMIA' if 'GASTRON' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'VETERINARIA Y ZOOTECNIA' if 'ZOOTECNIA' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'NUTRICION' if 'NUTRIC' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MARKETING' if ( 'MARKETING' in x or 'PUBLICIDAD' in x or 'MERCAD' in x) else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MATEMATICAS Y ESTADISTICA' if ('MATE' in x or 'ESTA' in x) else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ARTES' if 'ARTE' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'PERIODISMO' if 'PERIOD' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'LITERATURA' if 'LITER' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'MUSICA' if ('MUS' in x or 'MÚS' in x) else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ECONOMIA Y NEGOCIOS' if ('ECONO' in x or 'NEG' in x or 'COME' in x or 'FIN' in x) else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS DE LA TIERRA' if 'GEO' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS AUDIOVISUALES' if ('CINE' in x or 'RADIO' in x or 'TELEV' in x or 'AUDIOV' in x or 'FOTOG' in x) else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'DERECHO' if 'DERECHO' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS POLITICAS' if ( 'POLIT' in x or 'POLÍT' in x) else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INGENIERIA' if 'INGE' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'QUIMICA' if ('QUIM' in x or 'QUÍM' in x or 'QU¿M' in x) else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CONSTRUCCION' if 'CONST' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'TURISMO' if 'TURIS' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BIBLIOTECOLOGIA' if 'BIBLIO' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'FILOSOFIA' if 'FILOS' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'FILOLOGIA' if 'FILOL' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ODONTOLOGIA' if 'ODONTOLOG' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BIOLOGIA Y MICROBIOLOGIA' if 'BIOL' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'ECOLOGIA' if x.startswith('ECOL')   else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'BACTERIOLOGIA' if 'BACT' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS CULTURALES' if 'CULT' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'INSTRUMENTACION QUIRURGICA' if 'INSTRUMENTACION' in x else x)
  bd2['ESTU_PRGM_ACADEMICO'] = bd2['ESTU_PRGM_ACADEMICO'].apply(lambda x: 'CIENCIAS MARITIMAS Y AERONAUTICAS' if ('MILIT' in x or 'NAVALES' in x or 'NAU' in x) else x)

  bd2 = pd.get_dummies(bd2, columns = vcat, drop_first=True)

  return bd2

In [ ]:
# Seleccionar etiquetas de entrenamiento (ID y RENDIMIENTO_GLOBAL)
etiqentre = bdtrain[['ID', 'RENDIMIENTO_GLOBAL']].copy()

# Determinar las columnas predictoras (todas excepto RENDIMIENTO_GLOBAL)
colspredict = bdtrain.columns.drop('RENDIMIENTO_GLOBAL')

# Unir bdtrain y bdtest solo con las columnas predictoras
bd = pd.concat([bdtrain[colspredict], bdtest[colspredict]], ignore_index=True)

# Limpiar el conjunto combinado
bdpro = limpdata(bd)

# Convertir columnas booleanas a tipo entero (0 y 1)
bdpro = bdpro.apply(lambda col: col.astype(int) if col.dtype == 'bool' else col)

# Separar nuevamente los conjuntos de entrenamiento y prueba usando los IDs de etiqentre
ids_entrenamiento = etiqentre['ID']
bdtrain2 = bdpro[bdpro['ID'].isin(ids_entrenamiento)]
bdtest2 = bdpro[~bdpro['ID'].isin(ids_entrenamiento)]

# Añadir la variable objetivo al conjunto de entrenamiento procesado
trainlisto = bdtrain2.merge(etiqentre, on='ID', how='left')

# Calcular el tamaño del conjunto de entrenamiento procesado
tamanoent = len(trainlisto)

['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE']


/tmp/ipython-input-7-4067068945.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bd2[column].fillna(vc[column], inplace=True)
/tmp/ipython-input-7-4067068945.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bd2[col] = bd2[col].replace({'Si': 1, 'No': 0})
/tmp/ipython-input-7-4067068945.py:38

In [ ]:
bdpro

,ID,PERIODO,coef_1,coef_2,coef_3,coef_4,ESTU_PRGM_ACADEMICO_ADMINISTRACION,ESTU_PRGM_ACADEMICO_AGROINDUSTRIA,ESTU_PRGM_ACADEMICO_AGRONOMIA,ESTU_PRGM_ACADEMICO_AGRONOMIA DEL TROPICO HUMEDO,...,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Técnica o tecnológica completa,FAMI_EDUCACIONMADRE_Técnica o tecnológica incompleta
0,904256,20212,0.322,0.208,0.310,0.267,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,645256,20212,0.311,0.215,0.292,0.264,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,308367,20203,0.297,0.214,0.305,0.264,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,470353,20195,0.485,0.172,0.252,0.190,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,989032,20212,0.316,0.232,0.285,0.294,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989281,496981,20195,0.168,0.410,0.235,0.300,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
989282,209415,20183,0.471,0.184,0.264,0.193,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
989283,239074,20212,0.292,0.249,0.276,0.256,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
989284,963852,20195,0.305,0.219,0.310,0.260,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
label_encoder = LabelEncoder()
trainlisto['RENDIMIENTO_GLOBAL'] = label_encoder.fit_transform(trainlisto['RENDIMIENTO_GLOBAL'])

In [ ]:
# Definir las columnas que se usarán, excluyendo ID y RENDIMIENTO_GLOBAL
X = [col for col in trainlisto.columns if col not in ['ID', 'RENDIMIENTO_GLOBAL']]

# Crear un base de datos
Xtrain = trainlisto[X].copy()

# Extraer la variable objetivo
y = trainlisto['RENDIMIENTO_GLOBAL'].tolist()

# Función personalizada para dividir datos en entrenamiento y prueba
def dividir_datos(datx, daty, proppru=0.2, semilla=42):
    np.random.seed(semilla)
    indtot = np.arange(len(datx))
    indpru = np.random.choice(indtot, size=int(len(datx) * proppru), replace=False)
    indentr = np.setdiff1d(indtot, indpru)

    X_train = datx.iloc[indentr]
    X_test = datx.iloc[indpru]
    y_train = [daty[i] for i in indentr]
    y_test = [daty[i] for i in indpru]

    return X_train, X_test, y_train, y_test

# Dividir los datos usando la función personalizada
X_train, X_test, y_train, y_test = dividir_datos(
    Xtrain, y, proppru=0.2, semilla=42
)

In [ ]:
!pip install xgboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.3/322.3 MB 3.4 MB/s eta 0:00:00


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Modelo con Random Forest y XGBoost
XGmodel = XGBClassifier(
    n_estimators=100,             # Número de árboles (como Random Forest)
    learning_rate=1,              # Importante para no hacer boosting secuencial
    subsample=0.8,                # Fracción de muestras usadas por árbol
    colsample_bytree=0.8,         # Fracción de columnas usadas por árbol
    max_depth=6,                  # Profundidad máxima de los árboles
    num_parallel_tree=100,        # Igual que n_estimators
    booster='gbtree',
    objective='binary:logistic',  # Cambia a 'multi:softprob' para clasificación multiclase
    random_state=42,
    n_jobs=-1                     # Usar todos los núcleos
)

# Entrenar el modelo
XGmodel.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_parallel_tree=100, ...)

In [ ]:
prediccion = XGmodel.predict(X_test)
prec = accuracy_score(y_test, prediccion)

print("Random Forest Accuracy:", accuracy_score(y_test, prediccion))
print("Random Forest Reporte de Clasificacion:\n", classification_report(y_test, prediccion))

Random Forest Accuracy: 0.4303321299638989
Random Forest Reporte de Clasificacion:
               precision    recall  f1-score   support

           0       0.55      0.63      0.59     35165
           1       0.45      0.57      0.50     34573
           2       0.32      0.26      0.29     34259
           3       0.33      0.27      0.30     34503

    accuracy                           0.43    138500
   macro avg       0.41      0.43      0.42    138500
weighted avg       0.41      0.43      0.42    138500



In [ ]:
# Asegurarte de que estás usando solo las columnas correctas
new_data_processed = bdtest2[X_train.columns].copy()

# Rellenar valores faltantes si es necesario
new_data_processed = new_data_processed.fillna(0)

# Asegurar tipos numéricos
new_data_processed = new_data_processed.apply(pd.to_numeric, errors='coerce')

# Predecir
predict = XGmodel.predict(new_data_processed)

# Asignar predicción al dataframe
bdtest2 = bdtest2.copy()  # Para evitar advertencias
bdtest2['RENDIMIENTO_GLOBAL'] = predict

# Preparar salida
test = bdtest2[['ID', 'RENDIMIENTO_GLOBAL']].copy()

# Aplicar mapeo de etiquetas
mapping = {2: 'medio-alto', 0: 'alto', 1: 'bajo', 3: 'medio-bajo'}
test['RENDIMIENTO_GLOBAL'] = test['RENDIMIENTO_GLOBAL'].map(mapping)

# Guardar CSV
test.to_csv('99.csv', index=False)


In [ ]:
from google.colab import files
files.download('99.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>